In [11]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import joblib
import gspread
from google.oauth2.service_account import Credentials
from google.colab import auth
from google.auth import default

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

# Load the trained model
try:
    rf_classifier = joblib.load('model.pkl')
except FileNotFoundError:
    print("Error: Trained model file 'model.pkl' not found.")
    exit(1)

# Define worksheet keys for the two sheets
worksheet_key_data1 = '1uGkGDLiiUxCdcdTVYyeiUMeUSsoUyFl6cPbRjp0rJm4'
worksheet_key_data2 = '1uGkGDLiiUxCdcdTVYyeiUMeUSsoUyFl6cPbRjp0rJm4'
worksheet_key_soil = '1uGkGDLiiUxCdcdTVYyeiUMeUSsoUyFl6cPbRjp0rJm4'

# Access the data1 worksheet
wb_data1 = gc.open_by_key(worksheet_key_data1)
ws_data1 = wb_data1.worksheet('Sheet1')  # Modify the worksheet name as needed

# Retrieve the latest values from the data1 worksheet.
rows_data1 = ws_data1.get_all_values()
latest_data_data1 = rows_data1[-1]#[1:]  # Skip the header row

# Access the data2 worksheet
wb_data2 = gc.open_by_key(worksheet_key_data2)
ws_data2 = wb_data2.worksheet('Sheet2')  # Modify the worksheet name as needed

# Retrieve the latest values from the data2 worksheet.
rows_data2 = ws_data2.get_all_values()
latest_data_data2 = rows_data2[-1]

# Access the soil worksheet
wb_soil = gc.open_by_key(worksheet_key_soil)
ws_soil = wb_soil.worksheet('soiltypes')  # Modify the worksheet name as needed

# Retrieve the latest value from the soil worksheet.
rows_soil = ws_soil.get_all_values()
latest_soil_data = rows_soil[-1][0]

# Combine the latest data into a single list
merged_latest_data = latest_data_data1 + latest_data_data2 + [latest_soil_data]

# Print and store the merged latest data in a variable
print("Merged Latest Data:", merged_latest_data)

# Your trained model may expect feature names, so create a dictionary to map feature names to values.
# Replace 'feature_names' with your actual feature names.
feature_names = {
    'N': merged_latest_data[0],
    'P': merged_latest_data[1],
    'K': merged_latest_data[3],
    'temperature': merged_latest_data[5],
    'humidity': merged_latest_data[4],
    'ph': merged_latest_data[2],
    #'rainfall': merged_latest_data[6],
    'soil': merged_latest_data[6]
}

# Convert the feature names to a DataFrame
df_latest_data = pd.DataFrame([feature_names])

# Assuming 'predicted_probabilities' is an array of predicted probabilities for each crop class
predicted_probabilities = rf_classifier.predict_proba(df_latest_data)

crop_classes = rf_classifier.classes_

# Create a list of (crop, probability) tuples and sort them by probability in descending order
crop_probabilities = list(zip(crop_classes, predicted_probabilities[0]))
'''print(crop_probabilities)'''
sorted_crop_probabilities = sorted(crop_probabilities, key=lambda x: x[1], reverse=True)

# Get the top 3 predicted crops
top_3_crops = sorted_crop_probabilities[:3]

# Assuming 'ws' represents the worksheet you want to write to
worksheet_name = 'Sheet3'  # Change to the target worksheet name

# Update the target worksheet for writing
ws = wb_data1.worksheet(worksheet_name)
latest_data = latest_data_data1 + latest_data_data2
'''print(latest_data)'''
ferti = {
    'rice': ['Urea, CAN', 'Urea, CAN, MAP', 'Urea'],
    'maize': ['Urea, CAN, DAP', 'Urea, CAN, MAP', 'Urea, MOP'],
    'blackgram': ['Urea, CAN', 'Urea, CAN, DAP', 'Urea, MOP'],
    'mungbean': ['Urea, CAN', 'Urea, CAN, DAP', 'Urea, MOP'],
    'cotton': ['Urea, CAN', 'Urea, CAN, DAP, K2SO4, MgSO4', 'Urea, MOP, K2SO4, MgSO4'],
    'coffee': ['Urea, CAN', 'Urea, CAN, DAP, K2SO4, MgSO4', 'Urea, CAN, MOP, K2SO4, MgSO4'],
    'jute': ['Urea, CAN', 'Urea, CAN, DAP, K2SO4, MgSO4', 'Urea, MOP, K2SO4, MgSO4'],
    'coconut': ['Urea, CAN', 'Urea, CAN, DAP, K2SO4, MgSO4, Boron', 'Urea, MOP, K2SO4, MgSO4, Boron'],
    'kidneybeans': ['Urea, CAN', 'Urea, CAN, DAP, ZnSO4', 'Urea, MOP, ZnSO4'],
    'chickpea': ['Urea, CAN', 'Urea, CAN, DAP, ZnSO4', 'Urea, MOP, ZnSO4'],
    'pigeonpeas': ['Urea, CAN', 'Urea, CAN, DAP, ZnSO4', 'Urea, MOP, ZnSO4'],
    'lentil': ['Urea, CAN', 'Urea, CAN, DAP, ZnSO4', 'Urea, MOP, ZnSO4']
}


try:
    if latest_data == ['0', '0', '0', '0', '0', '0']:
    # Case 1: If the condition is satisfied, print "0 0"
        output_data = [['Crop', 'Probability'],[0, 0]]
        message = "0 0"
    else:
    # Case 2: If the condition is not satisfied, print the top 3 predicted crops
        print("\nTop 3 Predicted Crops:")
        '''for rank, (crop, probability) in enumerate(top_3_crops, start=1):
            print(f"{rank}. Crop: {crop}, Probability: {probability:.2f},")'''
        top = []
        for i in top_3_crops:
            top.append(list(i))
        for i in top:
          i.extend(ferti[i[0]])
        output_data = [['Crop', 'Probability','Germination','Vegetative','Reproductive']] + top
        message = "Top 3 Predicted Crops printed in sheet3"

# Define the range where you want to write the data
    ws.clear()
    cell_range = f'A1:E{len(output_data)}'

# Update the specified cell range with the data
    ws.update(cell_range, output_data)

# Print the message indicating what was written to sheet3
    print(f"Data written to '{worksheet_name}' in the Google Sheets spreadsheet. Message: {message}")
except Exception as e:
    print(f"Error writing to Google Sheets: {str(e)}")


Merged Latest Data: ['29', '41', '7.8', '83', '76', '27.96', '5']

Top 3 Predicted Crops:
Data written to 'Sheet3' in the Google Sheets spreadsheet. Message: Top 3 Predicted Crops printed in sheet3
